In [1]:
from particletracking import dataframes, statistics
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from scipy import spatial, signal
%matplotlib auto

Using matplotlib backend: Qt5Agg


/home/ppxjd3/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def gkern(kernlen=21, std=3):
    gkern1d = signal.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d

In [3]:
def create_view(points, std):
    r = int(points.r.mean())
    std = r * std
    k = gkern(2*r, std)
    grid = np.zeros((points.x.max(), points.y.max()))
    for i, p in points.iterrows():
        
        x = p.x
        y = p.y
        try:
            grid[x-r:x+r, y-r:y+r] += k
        except ValueError as e:
            pass
    return grid


In [4]:
def transform_view(view):
    transform = np.fft.fft2(view)
    transform = np.abs(transform)
    transform = np.log(transform) # Put the numbers on a nicer scale for viewing
    transform = np.fft.fftshift(transform)
    return transform

In [5]:
def get_peaks(view):
    peaks = peak_local_max(view, 20)
    tree = spatial.KDTree(peaks[1:])
    dist, indices = tree.query(peaks[0], 12)
    chosen_peak = peaks[indices[8]] # Indices 6 to 11 represent peaks of interest
    return peaks[0], chosen_peak

In [6]:
def get_vector_from_peaks(central, ring, spacing):
    vector = ring - central
    vector = vector * spacing
    return vector    

In [7]:
def primary_reciprocal_lattice_vector(points):
    points *= 10
    particle_view = create_view(points, 0.1)
    particle_view_transformed = transform_view(particle_view)
    central_peak, ring_peak = get_peaks(particle_view_transformed)
    spacing = 1/len(particle_view)
    vector = get_vector_from_peaks(central_peak, ring_peak, spacing)
    return vector
    

In [8]:
def get_torder(points, G):
    return np.exp(1j*np.dot(points[['x', 'y']].values, G))

In [9]:
def get_psis(df, param):
    vals = df[param].values
    
    return np.abs(np.mean(vals * np.conj(vals))), np.abs(np.mean(vals)*np.conj(np.mean(vals)))


def get_susceptibility(df, param):
    psi = np.array(df.groupby('frame').apply(get_psis, param).values)
    psi_2, psi = zip(*psi)
    sus = np.mean(psi_2) - np.mean(psi)**2
    return sus

In [10]:
def guess_G_length(points):
    r = points.r.mean()
    large_spacing = 4.8/2*r
    apothem = large_spacing / (2*np.sqrt(3)/3)
    return 2*np.pi / apothem

In [11]:
def guess_G(points):
    L = guess_G_length(points)
    angles = np.arange(1, 120) * np.pi / 180
    Gs = np.stack((np.cos(angles), np.sin(angles)), axis=0)*L
    return Gs

In [12]:
def mean_of_other_vectors(points, G):
    x = np.stack(np.meshgrid(points.x, points.x), axis=-1)
    y = np.stack(np.meshgrid(points.y, points.y), axis=-1)
    dx = x[:, :, 1] - x[:, :, 0]
    dy = y[:, :, 1] - y[:, :, 0]
    u = np.stack((dx, dy), axis=-1)
    step = np.exp(1j*u@G)
    torder = np.mean(step, axis=0)
    return torder
    

In [13]:
def get_both_sus(df):
    points = df.loc[30000]
    G_all = guess_G(points)
    torder_all = np.array([np.exp(1j*points[['x', 'y']]@G) for G in G_all.T])
    G = G_all[:, np.argmax(np.sum(torder_all, axis=1))]
    df['torder'] = np.exp(1j*df[['x', 'y']].values@G)
    df['order_long_complex'] = df['order_r_long'] + 1j*df['order_i_long']
    group = df.groupby('Duty')
    torder_sus = group.apply(get_susceptibility, 'torder')
    hex_order_sus = group.apply(get_susceptibility, 'order_long_complex')
    return torder_sus, hex_order_sus

In [14]:
with dataframes.DataStore("/media/data/Data/FirstOrder/PhaseDiagram/FlatPlate2Feb2021/2000.hdf5") as data:
    flat_df = data.df
    
flat_torder_sus, flat_horder_sus = get_both_sus(data.df)

In [14]:
with dataframes.DataStore('/media/data/Data/FirstOrder/PhaseDiagram/DimpledPlateFeb2021/2100.hdf5') as data:
    dimple_torder_sus, dimple_horder_sus = get_both_sus(data.df)

In [16]:
flat_horder_sus.plot()
flat_torder_sus.plot()

<AxesSubplot:xlabel='Duty'>

In [15]:
dimple_horder_sus.plot()
dimple_torder_sus.plot()

<AxesSubplot:xlabel='Duty'>

In [14]:
dimpled = dataframes.DataStore('/media/data/Data/FirstOrder/PhaseDiagram/DimpledPlateFeb2021/2100.hdf5')

MemoryError: Unable to allocate 3.21 GiB for an array with shape (8, 107591839) and data type float32